In [1]:
import glob #to read the files
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.interpolate import interp1d
import os.path 
from tabulate import tabulate #to export in table format

In [2]:
X_ = np.linspace(-10, 50, 60 + 1)
wave = np.linspace(2000, 9200, 720 + 1)
list_data = [[]for y in range(0,721)]
save_path = '/home/joao/Documentos/PCA'

In [3]:
file = open("names.txt")
lines = file.readlines()
file.close()

In [4]:
training_set = []

In [5]:
for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    training_set.append(co[0])

In [6]:
len(training_set)

23

In [7]:
for j in range(0,len(training_set)):

    sn_names=glob.glob(training_set[j]+"*"+".dat")
    
    print(training_set[j])
    for i in range(0,len(sn_names)):

        y = []
        z = []

        file = open(sn_names[i])
        lines = file.readlines()
        file.close()

        for line in lines:
            if line.startswith('#'): continue
            co=line.rstrip().replace('INDEF','Nan').split()

            y.append(co[0])
            z.append(co[1])


        y = np.array(y, dtype=float)
        z = np.array(z, dtype=float)

        timetemp = lines[0]
        timetemp = timetemp[:-1]
        timetemp = float(timetemp[1:])

        f = max(z)

        for jj in range(0,len(y)):

            list_data[jj].append([timetemp,z[jj]])

        #print(i)

    Export = []
    
    for jj in range(0,len(list_data)):

        yy = []
        zz = []

        for k in range(0,len(list_data[jj])):

            yy.append([list_data[jj][k][0]])
            zz.append(list_data[jj][k][1])

        ff = max(zz)

        zz = zz/ff

        kernel = 1.0 * RBF(length_scale=2*max(zz), length_scale_bounds=(0.01, 10)) + WhiteKernel(noise_level=1e-1, noise_level_bounds=(1e-10, 1e+1))

        gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(yy, zz)

        y_mean = gp.predict(X_[:, np.newaxis])

        for l in range(0,len(y_mean)):

            Export.append(y_mean[l]*ff)
            
            
    table = []
    for l in range(0,len(Export)):
        table.append((Export[l],training_set[j]))


    name_of_file = training_set[j]

    completeName = os.path.join(save_path, name_of_file+".dat")

    f = open(completeName, 'w')  
    f.write(tabulate(table, tablefmt="plain"))
    f.close()


SN2012fr


KeyboardInterrupt: 